In [ ]:
from find_leaks import LeakFinder
from network import RandomWaterDistributionNetwork
import gravis

leak_finder = LeakFinder()
random_network_maker = RandomWaterDistributionNetwork()

In [ ]:
number_of_nodes = 30
flow_input_at_source = 10000
number_of_leaks = 3

random_network = random_network_maker.random_network(
    number_of_nodes,
    flow_input_at_source,
    number_of_leaks
)

figure = gravis.d3(random_network)
figure.display(inline=True)

In [ ]:
leaks = leak_finder.find_leaks(random_network)
print(f"Found f{len(leaks)} in the network")
print("There are leaks at:")
leaks.sort(key=lambda leak: leak.flow_loss, reverse=True)
for leak in leaks:
    print(f"\t- Node {leak.from_node} with flow loss of ~{leak.flow_loss}")
    for edge in leak.potential_leak_edges():
        print(f"\t\t- Potentially on edge {edge}")

In [ ]:
node = 1
expected = leak_finder.get_flow_at(random_network, node)
actual = leak_finder.get_flow_out_total(random_network, node)

print(f"Expected flow out of node {1} to be {expected}")
print(f"It was actually {actual}")